In [1]:
from strands import Agent
from strands.models.ollama import OllamaModel
from strands import Agent, tool, ToolContext
import pprint 

model = OllamaModel(
    host="http://localhost:11434",
    model_id="llama3.1",
)


In [2]:
@tool(context=True)
def track_user_action(action: str, tool_context: ToolContext):
    """Track user actions in agent state.

    Args:
        action: The action to track
    """
    # Get current action count
    action_count = tool_context.agent.state.get("action_count") or 0

    # Update state
    tool_context.agent.state.set("action_count", action_count + 1)
    tool_context.agent.state.set("last_action", action)

    return f"Action '{action}' recorded. Total actions: {action_count + 1}"

@tool(context=True)
def get_user_stats(tool_context: ToolContext):
    """Get user statistics from agent state."""
    action_count = tool_context.agent.state.get("action_count") or 0
    last_action = tool_context.agent.state.get("last_action") or "none"

    return f"Actions performed: {action_count}, Last action: {last_action}"

In [3]:
agent = Agent(
    model=model,
    tools=[track_user_action, get_user_stats]
)



In [4]:
# Use tools that modify and read state
agent("Track that I logged in")
agent("Track that I viewed my profile")


Tool #1: track_user_action
 have successfully logged in. What would you like to do next?
Tool #2: track_user_action
 have successfully viewed your profile. What would you like to do next?

AgentResult(stop_reason='end_turn', message={'role': 'assistant', 'content': [{'text': 'You have successfully viewed your profile. What would you like to do next?'}]}, metrics=EventLoopMetrics(cycle_count=4, tool_metrics={'track_user_action': ToolMetrics(tool={'toolUseId': 'track_user_action', 'name': 'track_user_action', 'input': {'action': 'viewed_profile'}}, call_count=2, success_count=2, error_count=0, total_time=0.0008001327514648438)}, cycle_durations=[0.4097011089324951, 0.43019556999206543], agent_invocations=[AgentInvocation(cycles=[EventLoopCycleMetric(event_loop_cycle_id='e1755908-6bad-42f9-88f7-bb74f2c14195', usage={'inputTokens': 19, 'outputTokens': 197, 'totalTokens': 216}), EventLoopCycleMetric(event_loop_cycle_id='850de99e-dc91-4f07-a0bb-955d1c915d7e', usage={'inputTokens': 15, 'outputTokens': 102, 'totalTokens': 117})], usage={'inputTokens': 34, 'outputTokens': 299, 'totalTokens': 333}), AgentInvocation(cycles=[EventLoopCycleMetric(event_loop_cycle_id='a8985dec-264d-44

In [5]:
print(f"Actions taken: {agent.state.get('action_count')}")

Actions taken: 2


In [6]:
print(f"Last action: {agent.state.get('last_action')}")

Last action: viewed_profile


In [7]:
result = agent("Track that I viewed my profile message 3")


Tool #3: track_user_action
 have successfully viewed your profile messages (this is the third time). What would you like to do next?

In [8]:
pprint.pprint(result)

AgentResult(stop_reason='end_turn',
            message={'content': [{'text': 'You have successfully viewed your '
                                          'profile messages (this is the third '
                                          'time). What would you like to do '
                                          'next?'}],
                     'role': 'assistant'},
            metrics=EventLoopMetrics(cycle_count=6,
                                     tool_metrics={'track_user_action': ToolMetrics(tool={'input': {'action': 'viewed_profile_message'},
                                                                                          'name': 'track_user_action',
                                                                                          'toolUseId': 'track_user_action'},
                                                                                    call_count=3,
                                                                                    success_count=3